# Datan analysointi K2020_2 - Tehtävät kerta 3 - RATKAISUT

Palauta Optiman palautuslaatikkoon viimeistään su 16.2. Kirjoita palautuslaatikon kommenttikenttään, mitä tehtävät olet tehnyt kokonaan/osittain.

**Tee tehtävät ilman for-silmukoita tms.**

**Opiskelijan nimi:**

## 1. Maakunnat





Lue DataFrameen tiedosto jossa on tilastotietoja Suomen kunnnista (vuosilta 2017-19). Kenttäerotin puolipiste ja deimaalierotin pilkku.

https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt


a) Laske kuinka paljon maapinta-alaa on kussakin seutukunnassa ja tulosta maapinta-alaltaan 5 suurinta seutukuntaa suuruusjärjestyksessä.

b) Laske kuinka monta prosenttia kunkin maakunnan väestöstä asuu kaupungeissa (kuntamuoto) ja tulosta 5 suurinta %-osuutta. 
 
c) Laske ulkomaan ruotsinkielisten osuus maakunnittain (huom. et voi laskea prosenttiluvuista keskiarvoa, koska kuntien väkiluvut vaihtelevat). Tulosta 5 suurinta osuutta.



In [3]:
import pandas as pd

# luetaan tiedosto
df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt', sep=';', decimal=',')

# ryhmitellään seutukuntien mukaan ja lasketaan väkiluvuista summat
print(df.groupby('seutukunta')['maapinta-ala'].sum().sort_values(ascending=False).head(5))

print('\n-----------\n')

# ryhmitellään maakunnan mukaan, molemmat groupbyt antavat serieksen, joilla sama indeksi (maakuntien nimet), joten voidaan jakaa ne keskenään
print((df[df['kuntamuoto']=='Kaupunki'].groupby('maakunta')['Väkiluku'].sum()/df.groupby('maakunta')['Väkiluku'].sum()*100).sort_values(ascending=False).head(5))


print('\n-----------\n')

# lasketaan ensin ulkomaalaisten määrä kunnittain ja sitten jaetaan koko välikuvulla (ryhmiteltynä maakuntien mukaan)
df['ulk'] = df['Väkiluku']*df['Ruotsinkielisten osuus%']/100.0

print((df.groupby('maakunta')['ulk'].sum()/df.groupby('maakunta')['Väkiluku'].sum()*100).sort_values(ascending=False).head(5))



seutukunta
Pohjois-Lapin seutukunta    31893.91
Itä-Lapin seutukunta        20548.92
Tunturi-Lapin seutukunta    20512.50
Kehys-Kainuun seutukunta    13959.50
Rovaniemen seutukunta       11035.13
Name: maapinta-ala, dtype: float64

-----------

maakunta
Kymenlaakso        90.171177
Uusimaa            86.882115
Pirkanmaa          84.496840
Varsinais-Suomi    82.700352
Etelä-Pohjanmaa    81.707298
Name: Väkiluku, dtype: float64

-----------

maakunta
Ahvenanmaa         86.770785
Pohjanmaa          49.459568
Keski-Pohjanmaa     9.040979
Uusimaa             7.840641
Varsinais-Suomi     5.655749
dtype: float64


## 2.  access.log

Lue erään palvelimen (3 tunnin ajan) access.log-tiedosto (15 Mt zip, noin 600 000 riviä) DataFrameen  
http://gpspekka.kapsi.fi/accesslog2.zip  
2-kerran teoriamateriaalissa on ohjeita datan lukemiseen.

Kentät  
* ip-osoite
* ident-id (-)
* userid (-)
* time
* request
* status
* size
* referer
* user agent


Parsi aikakentästä tunti ja minuutit ja laske jokaiselle minuutille kuinka monta eri ip-osoitetta on kyseisenä päivänä sivulla vieraillut.   

Tulosta listaa suuruusjärjestyksessä eli eniten eri ip-osoitteita sisältänyt minuutti ensin. 

Alkua:
```
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760

```


In [25]:
import pandas as pd

df = pd.read_csv('http://gpspekka.kapsi.fi/accesslog.zip', 
                 sep = ' ', 
                 quotechar = '"',
                 usecols = [0,1,3,4,5,6,7,8,9], # eipä näitä kaikkia tässä tarvi
                 names = ['ip', 'ip2', 'date', 'timezone', 'request', 'status', 'size', 'referer', 'user_agent']
                )
 

df['minute'] = df['date'].str[13:18]  # poimitaan aika date-merkkijonosta

# nunique() antaa uniikkien arvojen määrät
print(df['ip'].groupby(df['minute']).nunique().sort_values(ascending=False).head(10))



minute
12:55    790
12:51    787
12:54    787
12:53    785
12:56    784
12:52    781
14:00    770
14:01    766
13:59    759
12:50    758
Name: ip, dtype: int64


## 3. Kone

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv  (11 Mt) on erään koneen toiminnasta dataa 10 sekunnin välein. Aikaleiman lisäksi datassa on sarake `katko`, joka on 1 kun koneen toiminta on ollut katkoksissa ja 0 kun kone on ollut toiminnassa.

Poimi datasta tietoa katkoista eli **peräkkäisistä 1-riveistä**. Listaa kustakin katkosta:  
  * monesko katko on kyseessä
  * koska katko alkoi (eli ensimmäinen aikaleima)
  * koska katko loppui (eli viimeinen aikaleima)
  * kauanko katko kesti tunteina (rivien määrä jaettuna 360:lla).
  
10 ensimmäistä katkoa pitäisi näyttää tältä:
![katkot](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.png)  

In [6]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv')

# jos katko-arvo on yhden suurempi kuin edellisellä rivillä
df['katkoalkaa'] = df['katko'].diff(1)==1

df['katko_nro'] = df['katkoalkaa'].cumsum()

dfkatkot = pd.DataFrame(df[df['katko']==1].groupby('katko_nro')['aika'].first())  # kunkin katkon ekan rivin aika

dfkatkot['loppui'] = df[df['katko']==1].groupby('katko_nro')['aika'].last() # kunkin katkon viimeinen aika

dfkatkot['kesto_h'] = df[df['katko']==1].groupby('katko_nro')['aika'].count()/360 # kunkin katkon kesto


dfkatkot.head(10)

,aika,loppui,kesto_h
katko_nro,,,
1,2019-03-18T11:26:30.000000Z,2019-03-18T11:34:30.000000Z,0.136111
2,2019-03-18T11:35:30.000000Z,2019-03-18T11:43:00.000000Z,0.127778
3,2019-03-18T14:05:30.000000Z,2019-03-18T14:16:50.000000Z,0.191667
4,2019-03-18T21:49:00.000000Z,2019-03-18T21:56:00.000000Z,0.119444
5,2019-03-18T22:51:20.000000Z,2019-03-18T22:56:20.000000Z,0.086111
6,2019-03-18T22:57:20.000000Z,2019-03-18T22:59:40.000000Z,0.041667
7,2019-03-19T01:47:40.000000Z,2019-03-19T02:05:30.000000Z,0.300000
8,2019-03-19T02:54:40.000000Z,2019-03-19T02:58:40.000000Z,0.069444
9,2019-03-19T03:54:00.000000Z,2019-03-19T04:00:00.000000Z,0.102778


## 4. Lumeton alkutalvi?

Lataa DataFrameen csv-tiedosto  
http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv

Tiedostossa on päivittäisiä säähavaintoja Jyväskylän lentoasemalta vuodesta 1959.

Laske tietojen perusteella jokaiselle "alkutalvelle" 1. marraskuuta - 18. tammikuuta  **"lumisumma"** jossa on alkutalven jokaisen vuorokauden lumensyvyys-lukema laskettu yhteen (ei ole mikään oikea meteorologinen termi).

Huomioita:
* lumisumma pitää laske talvittain, ei siis vuosittain. Tee uusi sarake joka kertoo mihin (alku)talveen kyseinen vuorokausi kuuluu. Siis tammikuu 2020 kuuluu samaan alkutalveen kuin joulukuu 2019.

* Ilmatieteen laitos käyttää arvoa -1 lumensyvyyden arvona jos "lunta ei ole näkyvissäkään". Ei haluta kuitenkaan miinustaa yhtä näinä päivinä, joten korvaa -1:t nollilla.

* Lisäksi joinain päivinä lumensyvyys todella (jostain syystä) puuttuu. Jos nämä jätetään laskusta pois, kyseisen talven lukema jää liian pieneksi (jos lunta on kuitenkin ollut). Lienee turvallista olettaa että lunta on varmaan ollut suunnilleen saman verran kuin edellisenä päivänä, joten täytä NaN-arvot edellsillä arvoilla

Anna tuloksena DataFrame, jossa on indeksinä talvi (joko muodossa 2019 tai 2019-2020) ja sarakkeina 
* alkutalventalven lumisumma 
* sijoitus lumisummien joukossa (lumisin alkutalvi = 1) 
* alkutalven lumipäivien lukumäärä (eli kuinka monena päivänä lumensyvyys on ollut suurempi kuin nolla)
* alkutalven suurin lumensyvyys.

mallia alusta ja lopusta:  
 ```
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
1959-1960         1292    20          61       38
1960-1961         1305    19          61       52
1961-1962         1116    28          54       41
1962-1963         1065    30          65       35
1963-1964          850    41          69       22
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
2015-2016          139    61          31       14
2016-2017          737    50          69       18
2017-2018         1476    15          62       49
2018-2019          568    53          39       31
2019-2020          856    39          68       30
 ```


In [5]:
import numpy as np
import pandas as pd


df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv')

# korvataan -1:t nollilla
df['Lumensyvyys'].replace(-1, 0, inplace = True)  

# täytetään puuttuvat edellisillä valideilla arvoilla
df['Lumensyvyys'].fillna(method = 'ffill', inplace = True) 

# suodatetaan vain alkutalvet
df = df[(df['Kk'].isin([11,12]) | ((df['Kk']==1) & (df['Pv']<19)))]


# Talvi edelliseen vuoteen jos on kuukausi 1
df['Talvi1'] = df['Vuosi'] - 1*(df['Kk']==1)   

# pidemmässä muodossa
df['Talvi'] = df['Talvi1'].astype(str) + '-' +  (df['Talvi1']+1).astype(str)

# ryhmitellään talven mukaan. Muutetaan Series dataframeksi, jotta voidaan lisätä sarakkeita myöhemmin
df2 = pd.DataFrame(df.groupby(df['Talvi'])['Lumensyvyys'].sum().astype(int))  

# sijoitukset rank():lla
df2['sija'] = df2['Lumensyvyys'].rank(ascending = False).astype(int) 

# lumipäivien määrä
df2['lumipaivia'] = df[df['Lumensyvyys']>0].groupby(df['Talvi'])['Lumensyvyys'].count()

# lumimaksimi
df2['maksimi'] = df.groupby(df['Talvi'])['Lumensyvyys'].max().astype(int)

print(df2.head(5))
print(df2.tail(5))




           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
1959-1960         1292    20          61       38
1960-1961         1305    19          61       52
1961-1962         1116    28          54       41
1962-1963         1065    30          65       35
1963-1964          850    41          69       22
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
2015-2016          139    61          31       14
2016-2017          737    50          69       18
2017-2018         1476    15          62       49
2018-2019          568    53          39       31
2019-2020          856    39          68       30


## 5. Liiga

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt on SM-liigan (nyk. Liiga) runkosarjan ottelutukset 2010-luvulta.  

huom. -sarakkeessa on merkintä JA tai VL, jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa.

Voittanut (enemmän maaleja tehnyt joukkue) saa 3 pistettä ja hävinnyt 0 pistettä, paitsi jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa, voittanut joukkue saa 2 pistettä ja hävinnyt yhden pisteen.



Generoi tulosten perusteella "vuosikymmenen sarjataulukko", jossa on kullekin joukkueelle omana sarakkeenaan
* sijoitus
* joukkueen nimi
* pelatut ottelut
* 3 pisteen voitot
* 2 pisteen voitot (JA/VL)
* 1 pisteen tappiot (JA/VL)
* 0 pisteen tappiot
* tehdyt maalit ja päästetyt maalit (samassa sarakkeessa)
* pisteet



Lajittele sarjataulukko pisteiden mukaan (eniten pisteitä ekana). Tasapisteissä olevat joukkueet menevät maalieron (tehdyt maalit - päästetyt maalit) mukaiseen järjestykseen, ja jos sekin on sama niin tehdyt maalit ratkaisevat.

tavoiteltu lopputulos:

```
    sija   joukkue  ott   v3  v2  t1   t0     maalit  pist
0      1    Kärpät  574  295  70  51  158  1678-1244  1076
1      2       JYP  575  277  62  58  178  1614-1311  1013
2      3   Tappara  575  261  78  62  174  1598-1387  1001
3      4      HIFK  575  264  62  63  186  1637-1403   979
4      5     KalPa  574  224  69  68  213  1414-1400   878
5      6     Lukko  574  226  62  67  219  1516-1479   869
6      7     Ässät  574  215  64  56  239  1428-1501   829
7      8  Pelicans  574  215  57  56  246  1530-1595   815
8      9       HPK  575  198  80  60  237  1416-1498   814
9     10     SaiPa  574  201  66  71  236  1451-1576   806
10    11       TPS  572  193  59  78  242  1410-1596   775
11    12     Ilves  574  192  55  60  267  1426-1643   746
12    13     Blues  360  126  43  36  155    852-943   500
13    14   Jokerit  240  102  29  37   72    672-586   401
14    15     Sport  334   87  37  47  163   810-1028   382
15    16    KooKoo  275   79  32  42  122    664-809   343
16    17   Jukurit  215   56  21  34  104    486-603   244
```

(jos saat Ässille 832 pistettä, ohjelmakoodisi tulkitsee voittajan tuloksesta 11-3 väärin...) 


In [4]:


import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt')

# luetaan joukkueet ja maalit. Maalit pitää muuttaa luvuiksi, jos ne jää merkkijonoiksi niin 11-3 -tulos menee vierasvoitoksi 
# koska merkkijonot vertaillaan aakkosjärjestyksessä

df['kotij'] = df['ottelu'].str.split('-').str[0]  
df['vierasj'] = df['ottelu'].str.split('-').str[1]
df['kotim'] = df['tulos'].str.split('-').str[0].astype(int)
df['vierasm'] = df['tulos'].str.split('-').str[1].astype(int)

# alustetaan pisteet nolliksi

df['kotipist'] = 0
df['vieraspist'] = 0

# pistelaskussa toimisi myös  vieraspisteet = 3 - kotipisteet

# varsinaisella peliajalla voitosta 3 pistettä
df.loc[(df['huom'].isnull()) & (df['kotim']>df['vierasm']), 'kotipist'] = 3
df.loc[(df['huom'].isnull()) & (df['kotim']<df['vierasm']), 'vieraspist'] = 3

# JA/VL - peleissä pisteet 2-1
df.loc[(df['huom'].notnull()) & (df['kotim']>df['vierasm']), 'kotipist'] = 2
df.loc[(df['huom'].notnull()) & (df['kotim']<df['vierasm']), 'kotipist'] = 1
df.loc[(df['huom'].notnull()) & (df['kotim']>df['vierasm']), 'vieraspist'] = 1
df.loc[(df['huom'].notnull()) & (df['kotim']<df['vierasm']), 'vieraspist'] = 2

# Seriekseinä kunkin joukkueen koti/vierasvoitot/tasapelit/tappiot ja maalit 
# kaikilla näille Serieksillä on sama indeksi (joukkueet), joten niitä voi laskea yhteen tms 

k3 = df[df['kotipist']==3].groupby(df['kotij'])['kotij'].count()  # 3p-kotivoitot
k2 =  df[df['kotipist']==2].groupby(df['kotij'])['kotij'].count()  # 2p-kotivoitot
k1 =  df[df['kotipist']==1].groupby(df['kotij'])['kotij'].count()  # 1p-kotitappiot
k0 = df[df['kotipist']==0].groupby(df['kotij'])['kotij'].count()  # 0p-kotitappiot


v3 = df[df['vieraspist']==3].groupby(df['vierasj'])['vierasj'].count()  # 3p-vierasvoitot
v2 =  df[df['vieraspist']==2].groupby(df['vierasj'])['vierasj'].count()  # 2p-vierasvoitot
v1 =  df[df['vieraspist']==1].groupby(df['vierasj'])['vierasj'].count()  # 1p-vierastappiot
v0 = df[df['vieraspist']==0].groupby(df['vierasj'])['vierasj'].count()  # 0p-vierastappiot

tm = df['kotim'].groupby(df['kotij']).sum() + df['vierasm'].groupby(df['vierasj']).sum()  # tehdyt maalit
pm = df['vierasm'].groupby(df['kotij']).sum() + df['kotim'].groupby(df['vierasj']).sum()  # päästeyt maalit   


# Seriekset DataFrameksi
# tämä edellyttää että kaikilla joukkueilla on kaikkia tuloksia k3-k0 ja v3-v0 mutta 10 vuodessa näin on
# muuten k3+v3 antaisi jollekin joukkueelle NaN jos jompi kumpi puuttuu

taul = pd.DataFrame({'v3':k3+v3, 'v2' :k2+v2, 't1' : k1+v1, 't0' : k0+v0, 'tm':tm, 'pm':pm})

# pisteet
taul['pist'] = taul['v3']*3+taul['v2']*2+taul['t1']

taul['maaliero'] = taul['tm']-taul['pm']
taul['ott'] = taul['v3'] + taul['v2'] + taul['t1'] + taul['t0']

# maalit merkkijonona
taul['maalit'] = taul['tm'].astype(str) + "-" + taul['pm'].astype(str) # muutetan stringeiksi

# lajitelukriteerit järjesyksessä (tosin tässä kaikilla oli eri pisteet)
taul.sort_values(['pist', 'maaliero', 'tm'], ascending = False, inplace = True)

# joukkuenimi on indeksinä (koska indeksi  on tullut groupby(df['kotij']) antamista Serieksistä)
# tässä tehdään siitä oma sarake
taul['joukkue'] = taul.index


# indeksiksi 0,1,2,3...
taul.reset_index(inplace=True)

# sija on indeksi + 1
taul['sija'] = taul.index+1

# otetaan vain tarvittavat sarakkeet
taul = taul[['sija', 'joukkue', 'ott', 'v3', 'v2', 't1', 't0', 'maalit', 'pist']]

print(taul)



    sija   joukkue  ott   v3  v2  t1   t0     maalit  pist
0      1    Kärpät  574  295  70  51  158  1678-1244  1076
1      2       JYP  575  277  62  58  178  1614-1311  1013
2      3   Tappara  575  261  78  62  174  1598-1387  1001
3      4      HIFK  575  264  62  63  186  1637-1403   979
4      5     KalPa  574  224  69  68  213  1414-1400   878
5      6     Lukko  574  226  62  67  219  1516-1479   869
6      7     Ässät  574  215  64  56  239  1428-1501   829
7      8  Pelicans  574  215  57  56  246  1530-1595   815
8      9       HPK  575  198  80  60  237  1416-1498   814
9     10     SaiPa  574  201  66  71  236  1451-1576   806
10    11       TPS  572  193  59  78  242  1410-1596   775
11    12     Ilves  574  192  55  60  267  1426-1643   746
12    13     Blues  360  126  43  36  155    852-943   500
13    14   Jokerit  240  102  29  37   72    672-586   401
14    15     Sport  334   87  37  47  163   810-1028   382
15    16    KooKoo  275   79  32  42  122    664-809   3